Conectado a Python 3.12.3

In [ ]:
from sklearn.model_selection import StratifiedKFold
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import f1_score
import numpy as np
import tensorflow as tf
from models import cnn_models as cnnmodel
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
    print("GPU(s) disponibles:", len(gpus))
else:
    print("No se encontraron GPUs disponibles.")

def normResults(y_pred):
    yp = []
    for row in y_pred:
        valmax = max(row)
        for i in range(len(row)):
            if row[i] == valmax:
                row[i] = 1
            else:
                row[i] = 0
        
        yp.append(row)
    
    return yp

def environment_model_graph(data1, name1, data2, name2, data3, name3, data4, name4, rootsave, nameSave = "graph", var_title="graph", xlabel="Axes X", ylabel="Axes Y"):
    num_epochs = len(data1[0])
    x = [i for i in range(num_epochs)]
    
    y1 = []
    y2 = []
    y3 = []
    y4 = []
    for i,j,m,n in zip(data1[0], data2[0], data3[0], data4[0]):
        y1.append(i)
        y2.append(j)
        y3.append(m)
        y4.append(n)
            
    sns.set(style="darkgrid")
    
    fig, axes = plt.subplots(2,1,figsize=(10,8))
    
    axes[0].plot(x, y1, label=name1)
    axes[0].plot(x, y2, label=name2)
    axes[0].set_title(var_title)
    axes[0].set_xlabel(xlabel)
    axes[0].set_ylabel(ylabel)
    axes[0].legend()
    
    axes[1].plot(x, y3, label=name3)
    axes[1].plot(x, y4, label=name4)
    axes[1].set_xlabel(xlabel)
    axes[1].set_ylabel(ylabel)
    axes[1].legend()
    
    plt.tight_layout()
    
    name = nameSave+'.png'
    plt.savefig(rootsave+name, dpi=300)

def add_column_df(df, kfold,precision, recall, f1_list, weighted_avg):
    
    df['kfold'] = [kfold, kfold]
    df['class'] = [0,1]
    df['precision'] = precision
    df['recall'] = recall
    df['f1'] = f1_list
    df['weighted_avg'] = [weighted_avg, weighted_avg]
    
    return df



"""Paths and data"""
root = '../experimental_test/'
root_graphs = root+'figures/'
#load data
data = np.load("../../../datasets/TCIA_SegPC_dataset/crops/dataset.npy")
labels = np.load("../../../datasets/TCIA_SegPC_dataset/crops/labels.npy")

data = data/255.0


""" Train and test """
skfolds = StratifiedKFold(n_splits=2)
columns = ['model', 'kfold', 'class', 'accuracy', 'precision', 'recall', 'f1', 'weighted_avg']
df = pd.DataFrame(columns=columns)
list_models = [cnnmodel.model_1(), cnnmodel.model_2(), cnnmodel.model_3(), cnnmodel.model_4(), cnnmodel.model_5(), 
               cnnmodel.model_6(), cnnmodel.model_7(), cnnmodel.model_8(), cnnmodel.model_9() ]


             
for nomodel in range(len(list_models)): 
    num_fold = 0
    model = list_models[nomodel]
    columns = ['kfold', 'phase', 'precision', 'recall', 'f1', 'weighted_avg']
    df = pd.DataFrame(columns=columns)
    df_parcial = pd.DataFrame(columns=columns)
    for train_index, test_index in skfolds.split(data, labels):
              
        #divide data
        X_train_folds = data[train_index]
        y_train_folds = labels[train_index]
        X_test_folds  = data[test_index]
        y_test_folds  = labels[test_index]
            
        
        y_train_folds = to_categorical(y_train_folds)
        y_test_folds = to_categorical(y_test_folds)
        
        """ train"""
        #define callback EarlyStopping
        early_stopping = EarlyStopping(monitor='val_loss', patience=100, verbose=1, restore_best_weights=True)
        #fit model
        history = model.fit(X_train_folds, y_train_folds, validation_data = (X_test_folds, y_test_folds), epochs=500, batch_size=32, callbacks=[early_stopping])

        accuracy_model=[]
        accuracy_model.append(model.evaluate(X_test_folds, y_test_folds))
        y_pred = model.predict(X_test_folds)          

        yp = [] 
        yp = normResults(y_pred)

        results = []
        n_correct = sum(yp == y_test_folds)
        results.append(n_correct/len(y_pred))
        
        
        yp2 = np.array(yp)
        precision = []
        recall = []
        f1_list = []
        for i in range(2):
            precision.append(precision_score(y_test_folds[:,i],yp2[:,i]))
            recall.append(recall_score(y_test_folds[:,i],yp2[:,i]))
            f1 = f1_score(y_test_folds[:,i],yp2[:,i])
            f1_list.append(f1)
    

        weighted_avg = []
        f1_list = np.array(f1_list)    
        weight = np.array(list(sum(y_test_folds == 1)))
        weighted_sum = np.sum(f1_list * weight)
        total_weight = np.sum(weight)
        weighted_avg.append(weighted_sum / total_weight)
        
        #df = add_column_df(df,nomodel, num_fold, accuracy_model, precision, recall, f1_list, weighted_sum / total_weight)

        num_fold += 1
        df_parcial = add_column_df(df_parcial,num_fold, precision, recall, f1_list, weighted_sum / total_weight)
        df = pd.concat([df, df_parcial])

    root_df = root+"sheets/"
    name_excel = "result_model_" + str(nomodel) + ".xlsx"
    df.to_excel(root_df + name_excel, index=False)

No se encontraron GPUs disponibles.


e:\python\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/500
39/39 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - accuracy: 0.6885 - loss: 0.6077 - val_accuracy: 0.8274 - val_loss: 0.4334
Epoch 2/500
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.7768 - loss: 0.4687 - val_accuracy: 0.8114 - val_loss: 0.4385
Epoch 3/500
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.8339 - loss: 0.3746 - val_accuracy: 0.7841 - val_loss: 0.4916
Epoch 4/500
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.8472 - loss: 0.3434 - val_accuracy: 0.8074 - val_loss: 0.4202
Epoch 5/500
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.8747 - loss: 0.2938 - val_accuracy: 0.8307 - val_loss: 0.4163
Epoch 6/500
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.8971 - loss: 0.2619 - val_accuracy: 0.7729 - val_loss: 0.4726
Epoch 7/500
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.8901 - loss: 0.2719 - val_accuracy: 0.7986 - val_loss: 0.4155
Epoch 8/500
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.8958 - loss: 0.2497 - val_accuracy: 0.

<ipython-input-1-5906e56caf3b>:162: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, df_parcial])


39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.7981 - loss: 0.4623 - val_accuracy: 0.8402 - val_loss: 0.4175
Epoch 2/500
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.7838 - loss: 0.4407 - val_accuracy: 0.8426 - val_loss: 0.3750
Epoch 3/500
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.8315 - loss: 0.3627 - val_accuracy: 0.8394 - val_loss: 0.3216
Epoch 4/500
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.8498 - loss: 0.3211 - val_accuracy: 0.9068 - val_loss: 0.2460
Epoch 5/500
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.8552 - loss: 0.2714 - val_accuracy: 0.9052 - val_loss: 0.2283
Epoch 6/500
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.8979 - loss: 0.2326 - val_accuracy: 0.9068 - val_loss: 0.2286
Epoch 7/500
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.9128 - loss: 0.2062 - val_accuracy: 0.9028 - val_loss: 0.2250
Epoch 8/500
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.9315 - loss: 0.1781 - val_accuracy: 0.9060 - val_l

...

39/39 ━━━━━━━━━━━━━━━━━━━━ 157s 4s/step - accuracy: 0.8304 - loss: 0.3622 - val_accuracy: 0.8578 - val_loss: 0.3872
Epoch 2/500
39/39 ━━━━━━━━━━━━━━━━━━━━ 156s 4s/step - accuracy: 0.7967 - loss: 0.4490 - val_accuracy: 0.8763 - val_loss: 0.2885
Epoch 3/500
39/39 ━━━━━━━━━━━━━━━━━━━━ 157s 4s/step - accuracy: 0.8737 - loss: 0.3217 - val_accuracy: 0.8948 - val_loss: 0.2812
Epoch 4/500
39/39 ━━━━━━━━━━━━━━━━━━━━ 155s 4s/step - accuracy: 0.8801 - loss: 0.3072 - val_accuracy: 0.8747 - val_loss: 0.2880
Epoch 5/500
39/39 ━━━━━━━━━━━━━━━━━━━━ 156s 4s/step - accuracy: 0.8446 - loss: 0.3290 - val_accuracy: 0.8980 - val_loss: 0.2583
Epoch 6/500
39/39 ━━━━━━━━━━━━━━━━━━━━ 154s 4s/step - accuracy: 0.8935 - loss: 0.2580 - val_accuracy: 0.9100 - val_loss: 0.2509
Epoch 7/500
39/39 ━━━━━━━━━━━━━━━━━━━━ 155s 4s/step - accuracy: 0.8707 - loss: 0.2991 - val_accuracy: 0.9124 - val_loss: 0.2172
Epoch 8/500
39/39 ━━━━━━━━━━━━━━━━━━━━ 156s 4s/step - accuracy: 0.9000 - loss: 0.2280 - val_accuracy: 0.9044 - val_l

<ipython-input-1-5906e56caf3b>:162: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, df_parcial])


39/39 ━━━━━━━━━━━━━━━━━━━━ 153s 4s/step - accuracy: 0.8864 - loss: 0.2941 - val_accuracy: 0.9068 - val_loss: 0.2310
Epoch 2/500
39/39 ━━━━━━━━━━━━━━━━━━━━ 156s 4s/step - accuracy: 0.8613 - loss: 0.3143 - val_accuracy: 0.9261 - val_loss: 0.2029
Epoch 3/500
39/39 ━━━━━━━━━━━━━━━━━━━━ 154s 4s/step - accuracy: 0.8950 - loss: 0.3390 - val_accuracy: 0.7912 - val_loss: 0.4995
Epoch 4/500
39/39 ━━━━━━━━━━━━━━━━━━━━ 154s 4s/step - accuracy: 0.8045 - loss: 0.4364 - val_accuracy: 0.8851 - val_loss: 0.2775
Epoch 5/500
39/39 ━━━━━━━━━━━━━━━━━━━━ 153s 4s/step - accuracy: 0.8544 - loss: 0.3357 - val_accuracy: 0.8819 - val_loss: 0.2793
Epoch 6/500
39/39 ━━━━━━━━━━━━━━━━━━━━ 158s 4s/step - accuracy: 0.8828 - loss: 0.2985 - val_accuracy: 0.7767 - val_loss: 0.3914
Epoch 7/500
39/39 ━━━━━━━━━━━━━━━━━━━━ 153s 4s/step - accuracy: 0.8210 - loss: 0.4534 - val_accuracy: 0.8434 - val_loss: 0.3452
Epoch 8/500
39/39 ━━━━━━━━━━━━━━━━━━━━ 156s 4s/step - accuracy: 0.8654 - loss: 0.3372 - val_accuracy: 0.8000 - val_l

<ipython-input-1-5906e56caf3b>:162: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, df_parcial])


39/39 ━━━━━━━━━━━━━━━━━━━━ 178s 5s/step - accuracy: 0.8037 - loss: 0.4672 - val_accuracy: 0.7574 - val_loss: 0.5279
Epoch 2/500
39/39 ━━━━━━━━━━━━━━━━━━━━ 178s 5s/step - accuracy: 0.7265 - loss: 0.6179 - val_accuracy: 0.7671 - val_loss: 0.4640
Epoch 3/500
39/39 ━━━━━━━━━━━━━━━━━━━━ 180s 5s/step - accuracy: 0.7608 - loss: 0.5148 - val_accuracy: 0.8305 - val_loss: 0.3973
Epoch 4/500
39/39 ━━━━━━━━━━━━━━━━━━━━ 182s 5s/step - accuracy: 0.7998 - loss: 0.4446 - val_accuracy: 0.7430 - val_loss: 0.4327
Epoch 5/500
39/39 ━━━━━━━━━━━━━━━━━━━━ 179s 5s/step - accuracy: 0.8196 - loss: 0.4139 - val_accuracy: 0.8635 - val_loss: 0.3415
Epoch 6/500
39/39 ━━━━━━━━━━━━━━━━━━━━ 177s 5s/step - accuracy: 0.8244 - loss: 0.4187 - val_accuracy: 0.8570 - val_loss: 0.3475
Epoch 7/500
39/39 ━━━━━━━━━━━━━━━━━━━━ 175s 5s/step - accuracy: 0.8705 - loss: 0.3148 - val_accuracy: 0.8257 - val_loss: 0.7250
Epoch 8/500
39/39 ━━━━━━━━━━━━━━━━━━━━ 173s 4s/step - accuracy: 0.8451 - loss: 0.4531 - val_accuracy: 0.7880 - val_l